In [23]:
import os, requests, time
import pandas as pd
import pyodbc
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [24]:
os.makedirs(os.path.join("Real Property Dataset"),exist_ok = True)

In [25]:
def Scrape_Data(driver): 
    House_Information = []
    Page = driver.page_source
    Source = BeautifulSoup(Page, "lxml")

    Titles = Source.find_all("span", class_="pr-title js__card-title")
    Areas = Source.find_all("span", class_="re__card-config-area js__card-config-item")
    Bedrooms = Source.find_all("span", class_="re__card-config-bedroom js__card-config-item")
    Bathrooms = Source.find_all("span", class_="re__card-config-toilet js__card-config-item")
    Location = Source.find_all("div", class_="re__card-location")
    Prices = Source.find_all("span", class_="re__card-config-price js__card-config-item")
    
    for title, area, bedroom, bathroom, price, local in zip(Titles, Areas, Bedrooms, Bathrooms, Prices, Location):
        House_Information.append({
            'House Title': title.text.strip(),
            'Area': area.text.strip(),
            'Bedrooms': bedroom.find('span').text.strip(),
            'Bathrooms': bathroom.find('span').text.strip(),
            'Location': local.text.strip(),
            'Price': price.text.strip()
        })
    
    return House_Information

In [26]:

def Browser_Automation():
    driver = webdriver.Edge()
    driver.set_window_size(1280, 720)
    URL = "https://batdongsan.com.vn/ban-nha-rieng-tp-hcm?cIds=325,163"
    driver.get(URL)
    
    House_List = []
    
    def close_popups():
        while True:
            try:
                close_button = WebDriverWait(driver, 0.5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "span.close")))
                close_button.click()
                time.sleep(0.25)
            except Exception:
                break
            
    while True:
        try:
            close_popups() 
            House_Data = Scrape_Data(driver)
            House_List.extend(House_Data)
            Next_Page = WebDriverWait(driver, 1.5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.re__pagination-icon i.re__icon-chevron-right--sm")))
            Next_Page.click()
            time.sleep(1)                                   
        except Exception:
            break
            
    driver.quit()
    return House_List

Scrape_House_List = pd.DataFrame(Browser_Automation())
Scrape_House_List.to_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"), header=True, index=False, encoding="utf-8-sig")

In [31]:
def Data_Cleaning():
    House_List = pd.read_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"))

    House_List["House Title"] = House_List["House Title"].str.replace('"', "")
    House_List["Area"] = House_List["Area"].str.replace("m²", "")
    House_List["Location"] = House_List["Location"].str.replace("·\n", "")
    House_List["Price"] = House_List["Price"].str.replace("tỷ", "").str.replace(",", ".")

    House_List.to_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"), header=True, index=False, encoding="utf-8-sig")

Data_Cleaning()

In [32]:
House_List = pd.read_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"))
House_List

,House Title,Area,Bedrooms,Bathrooms,Location,Price
0,Biệt thự 2 mặt tiền Nguyễn Đổng Chi: Cuộc sống...,302,5,5,"Quận 7, Hồ Chí Minh",Giá thỏa thuận
1,Lên sóng căn siêu phẩm ngay chợ HTT đường Lê V...,24,2,2,"Gò Vấp, Hồ Chí Minh",3.2
2,Siêu phẩm đẹp say đắm trệt lửng 2 lầu 3PN có g...,50,3,4,"Gò Vấp, Hồ Chí Minh",6.99
3,"Bán nhà phố 2 mặt view, ngay QL 13, Thủ Đức, g...",68,4,6,"Phường 2, Quận 3",6.4
4,Dinh thự ven sông biệt lập The Mizuki DT từ 53...,647,5,7,"Linh Xuân, Thủ Đức",63
...,...,...,...,...,...,...
27498,"Bán nhà ở tại đường Lý Thánh Tông, Tân Thới Hò...","88,95",1,2,"Tân Phú, Hồ Chí Minh",6.2
27499,Ngân hàng thanh lý nhà phố phường Tân Chánh Hi...,"127,3",3,3,"Cầu Ông Lãnh, Quận 1",5.4
27500,"Ngân hàng thanh lý nhà phố đường Bùi Quang Là,...","104,9",2,1,"Phường 14, Quận 10",7.3
27501,"Ngân hàng thanh lý nhà 4 tầng tại Quang Trung,...","66,3",4,5,"Phú Mỹ, Quận 7",5.6


In [ ]:
# House_List = pd.read_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"))
# Connection = pyodbc.connect("Driver={SQL Server};" 
#                             "Server=QUYSEGGS;" 
#                             "Database=tempdb;"
#                             "Trusted_Connection=yes;")
# Column_Name = House_List.columns.tolist()

# Cursor = Connection.cursor()

# Table_Name = "Real Property Information"

# Create_Table = f"create table {Table_Name} ("

# for Column in Column_Name:
#     if House_List[Column].dtype == 'int64':
#         Create_Table += f"{Column} INT, "
#     elif House_List[Column].dtype == 'float64':
#         Create_Table += f"{Column} FLOAT, "
#     else:
#         Create_Table += f"{Column} NVARCHAR(255), "
        
# # Loại bỏ dấu phẩy cuối cùng và đóng ngoặc
# Create_Table = Create_Table.rstrip(", ") + ")"

# Cursor.execute(Create_Table)
# Connection.commit()

# Insert_Cmd = f"INSERT INTO {Table_Name} ({', '.join(Column_Name)}) VALUES ({', '.join(['?' for _ in Column_Name])})"
# # Lặp qua từng hàng trong DataFrame và chèn dữ liệu vào bảng
# for index, row in House_List.iterrows():
#     Cursor.execute(Insert_Cmd, tuple(row))

# Connection.commit()

# Cursor.close()
# Connection.close()